## MNIST를 RNN으로

RNN 기본 개념

![RNN 기본 개념](RNN.png)

- 한 덩어리의 신경망을 셀이라 칭함
- RNN은 이 셀을 여러개 중첩하여 심층 신경망을 생성(앞 단계에서 학습한 결과를 다음 단계에 이용)
- 학습 데이터를 단계별로 구분하여 입력해야 함

In [3]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../mnist/data/", one_hot=True)

# hyper parameter
learning_rate = 0.001
total_epoch = 30
batch_size = 128

n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

# 입력/출력
# 순서가 있는 데이터이기 때문에 입력받을 개수와 총 몇단계로 이루어진 데이터를 받을지 결정 필요
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
# 가중치/편차
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

Extracting ../mnist/data/train-images-idx3-ubyte.gz
Extracting ../mnist/data/train-labels-idx1-ubyte.gz
Extracting ../mnist/data/t10k-images-idx3-ubyte.gz
Extracting ../mnist/data/t10k-labels-idx1-ubyte.gz


In [4]:
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [5]:
# 신경망 생성
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

In [7]:
# 결과값
model = tf.matmul(outputs, W) + b

In [8]:
# 손실값 및 최적화 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(total_epoch):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_step, n_input])
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y:batch_ys})
        
        total_cost += cost_val
        
    print('Epoch : %04d' % (epoch +1), 'Avg cost = {:.3f}'.format(total_cost / total_batch))
    
print('최적화 완료')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)

test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도 : ', sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))

Epoch : 0001 Avg cost = 0.554
Epoch : 0002 Avg cost = 0.255
Epoch : 0003 Avg cost = 0.187
Epoch : 0004 Avg cost = 0.167
Epoch : 0005 Avg cost = 0.148
Epoch : 0006 Avg cost = 0.132
Epoch : 0007 Avg cost = 0.125
Epoch : 0008 Avg cost = 0.117
Epoch : 0009 Avg cost = 0.112
Epoch : 0010 Avg cost = 0.106
Epoch : 0011 Avg cost = 0.095
Epoch : 0012 Avg cost = 0.100
Epoch : 0013 Avg cost = 0.095
Epoch : 0014 Avg cost = 0.091
Epoch : 0015 Avg cost = 0.090
Epoch : 0016 Avg cost = 0.080
Epoch : 0017 Avg cost = 0.086
Epoch : 0018 Avg cost = 0.075
Epoch : 0019 Avg cost = 0.084
Epoch : 0020 Avg cost = 0.077
Epoch : 0021 Avg cost = 0.082
Epoch : 0022 Avg cost = 0.071
Epoch : 0023 Avg cost = 0.072
Epoch : 0024 Avg cost = 0.070
Epoch : 0025 Avg cost = 0.076
Epoch : 0026 Avg cost = 0.065
Epoch : 0027 Avg cost = 0.062
Epoch : 0028 Avg cost = 0.061
Epoch : 0029 Avg cost = 0.069
Epoch : 0030 Avg cost = 0.065
최적화 완료
정확도 :  0.9762


## 단어 자동 완성

In [10]:
import tensorflow as tf
import numpy as np

char_arr = ['a', 'b', 'c', 'd', 'e','f', 'g',
           'h', 'i', 'j', 'k', 'l', 'm', 'n',
           'o', 'p', 'q', 'r', 's', 't', 'u',
           'v', 'w', 'x', 'y','z']

num_dic = {n: i for i , n in enumerate(char_arr)}
dic_len = len(num_dic)

In [11]:
# 학습 데이터 저장
seq_data = ['word', 'wood', 'deep', 'dive', 'cold' ,'cool', 'load', 'love', 'kiss', 'kind']

In [12]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
    return input_batch, target_batch

In [13]:
# 신경망 모델 구성
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

n_step = 3
n_input = n_class = dic_len

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [14]:
# RNN 셀 생성
cell1= tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [15]:
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [16]:
outputs = tf.transpose(outputs, [1, 0,2 ])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

In [17]:
# 손실함수, 최적화
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [18]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    
    print('Epoch: %04d' %(epoch +1), 'cost = {:.6f}'.format(loss))
print('최적화 완료')

Epoch: 0001 cost = 3.756218
Epoch: 0002 cost = 2.624926
Epoch: 0003 cost = 1.416641
Epoch: 0004 cost = 1.253464
Epoch: 0005 cost = 0.768952
Epoch: 0006 cost = 0.521900
Epoch: 0007 cost = 0.771612
Epoch: 0008 cost = 0.650361
Epoch: 0009 cost = 0.394681
Epoch: 0010 cost = 0.357200
Epoch: 0011 cost = 0.393904
Epoch: 0012 cost = 0.259976
Epoch: 0013 cost = 0.186685
Epoch: 0014 cost = 0.115431
Epoch: 0015 cost = 0.114479
Epoch: 0016 cost = 0.437807
Epoch: 0017 cost = 0.131866
Epoch: 0018 cost = 0.270023
Epoch: 0019 cost = 0.086541
Epoch: 0020 cost = 0.028202
Epoch: 0021 cost = 0.020861
Epoch: 0022 cost = 0.074780
Epoch: 0023 cost = 0.178056
Epoch: 0024 cost = 0.029700
Epoch: 0025 cost = 0.034167
Epoch: 0026 cost = 0.064535
Epoch: 0027 cost = 0.050619
Epoch: 0028 cost = 0.013120
Epoch: 0029 cost = 0.041470
Epoch: 0030 cost = 0.014167
최적화 완료


In [19]:
# 예측단어와 정확도
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

In [20]:
# 예측모델 실행
input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X: input_batch, Y: target_batch})

In [24]:
# 예측 단어 출력
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('예측 결과')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)

예측 결과
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0


## Sequence to Sequence(Seq2Seq)

![Sequence to Sequence 개념도](Seq2Seq.PNG)

- 입력을 위한 신경망인 인코더(Encoder)와 출력을 위한 신경망인 디코더(Decoder)로 구성
- 인코더는 원문, 디코더는 인코더가 번역한 결과물을 받음
- 디코더에 입력의 시작을 알리는 심볼(그림에서의 sos), 디코더의 출력이 끝났음을 알리는 심볼(그림에서의 eos)이 있음

In [1]:
import tensorflow as tf
import numpy as np

char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'], ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]

In [2]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S'+seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]
        
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
        
    return input_batch, output_batch, target_batch

In [3]:
# hyper parameter
learning_rate = 0.01
n_hidden = 128
total_epoch = 100

n_class = n_input = dic_len

In [4]:
# 신경망 구성
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

In [5]:
# RNN cell 구성
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)
    
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    
    # encoder의 계산 상태를 그대로 전달 위한 옵션(initial_state)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32) 

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
# 출력층 및 손실함수, 최적화 함수
model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [7]:
# 학습 진행
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    
    print('Epoch: %04d' % (epoch + 1), 'cost = {:.6f}'.format(loss))
    
print('최적화 완료')

Epoch: 0001 cost = 3.761093
Epoch: 0002 cost = 2.695197
Epoch: 0003 cost = 1.636204
Epoch: 0004 cost = 1.109002
Epoch: 0005 cost = 0.865946
Epoch: 0006 cost = 0.393956
Epoch: 0007 cost = 0.211484
Epoch: 0008 cost = 0.146563
Epoch: 0009 cost = 0.135851
Epoch: 0010 cost = 0.131395
Epoch: 0011 cost = 0.160218
Epoch: 0012 cost = 0.158918
Epoch: 0013 cost = 0.170710
Epoch: 0014 cost = 0.143665
Epoch: 0015 cost = 0.093192
Epoch: 0016 cost = 0.364346
Epoch: 0017 cost = 0.114636
Epoch: 0018 cost = 0.058626
Epoch: 0019 cost = 0.104609
Epoch: 0020 cost = 0.050559
Epoch: 0021 cost = 0.055761
Epoch: 0022 cost = 0.041746
Epoch: 0023 cost = 0.010906
Epoch: 0024 cost = 0.016942
Epoch: 0025 cost = 0.033720
Epoch: 0026 cost = 0.073835
Epoch: 0027 cost = 0.004760
Epoch: 0028 cost = 0.039932
Epoch: 0029 cost = 0.008085
Epoch: 0030 cost = 0.017432
Epoch: 0031 cost = 0.003691
Epoch: 0032 cost = 0.005032
Epoch: 0033 cost = 0.012118
Epoch: 0034 cost = 0.016693
Epoch: 0035 cost = 0.012253
Epoch: 0036 cost = 0

In [18]:
# 예측함수
def translate(word):
    seq_data = [word, 'P', len(word)]
    input_batch, output_batch, target_batch = make_batch([seq_data])
    
    # 예측 모델
    prediction = tf.argmax(model, 2)
    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    
    decoded = [char_arr[i] for i in result[0]]

    end = 0
    try:
        end = decoded.index('E')
    except ValueError as e:
        end = len(decoded)
    
    translated = ''.join(decoded[:end])
    return translated

In [20]:
print('===== 번역테스트======')
print('word -> ', translate('word'))
print('wodr -> ', translate('wodr'))
print('love -> ', translate('love'))
print('loev -> ', translate('loev'))
print('abcd -> ', translate('abcd'))

===== 번역테스트======
word ->  단어
wodr ->  나무
love ->  사랑
loev ->  사랑
abcd ->  단사


[참고 : Seq2Seq in Wiki](https://wikidocs.net/24996)